In [ ]:
# Install Cohere for embeddings, Umap to reduce embeddings to 2 dimensions, 
# Altair for visualization, Annoy for approximate nearest neighbor search
!pip install cohere umap-learn altair annoy datasets tqdm

In [ ]:
pip install umap

In [ ]:
pip install cohere

In [ ]:
pip install altair

In [ ]:
pip install annoy

In [ ]:
import cohere
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [ ]:
pip install annoy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


You'll need your API key for this next cell. [Sign up to Cohere](https://os.cohere.ai/) and get one if you haven't yet.

In [ ]:
# Paste your API key here. Remember to not share publicly
api_key = ''

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

## 1. Get The Archive of Questions
We'll use the [trec](https://www.tensorflow.org/datasets/catalog/trec) dataset which is made up of questions and their categories.

In [ ]:
# # Get dataset
# dataset = load_dataset("trec", split="train")

# # Import into a pandas dataframe, take only the first 1000 rows
# df = pd.DataFrame(dataset)[:1000]

# # Preview the data to ensure it has loaded correctly
# df.head(10)

import pandas as pd

# Get dataset
# dataset = load_dataset("trec", split="train")
# https://www.shanelynn.ie/pandas-csv-error-error-tokenizing-data-c-error-eof-inside-string-starting-at-line/
df = pd.read_excel("/content/news_articles_dataset.xlsx")

In [ ]:
df.head()

,Title,News
0,వాల్తేరు వీరయ్యలో రవితేజ రోల్‌ లీక్: అందులో చిరంజీవినే మించేలా!,తెలుగు సినీ ఇండస్ట్రీలో దాదాపు నలభై ఏళ్లుగా హవాను చూపిస్తూ స్టార్‌గా వెలుగొందుతోన్నారు మెగాస్టార్ చిరంజీవి. మధ్యలో కొంత కాలం గ్యాప్ ఇచ్చినా.. 'ఖైదీ నెంబర్ 150' మూవీతో సినీ రంగంలోకి రీఎంట్రీ ఇచ్చారు. అప్పటి నుంచి ఆయన వెనక్కి తిరిగి చూడకుండా వరుస సినిమాలతో దూసుకుపోతోన్నారు. ఇలా ఇప్పటికే పలు చిత్రాలతో ఈ మెగా హీరో ప్రేక్షకులను అలరించారు. ఈ క్రమంలోనే ఇప్పుడు మెగాస్టార్ చిరంజీవి 'వాల్తేరు వీరయ్య' అనే సినిమాను చేస్తున్నారు. మాస్ ఎంటర్‌టైనర్‌గా రాబోతున్న ఈ మూవీపై అంచనాలు భారీగానే ఉన్నాయి.
1,Balakrishna ఫ్యాన్స్ కు బాలయ్య మాస్ ట్రీట్.. ఒక్క టికెట్ పై ముగ్గురిని తలపించేలా డిఫరెంట్ గా!,"ఇటీవల అఖండ సినిమాతో సాలిడ్ హిట్ కొట్టిన నందమూరి నటసింహం బాలయ్య బాబు మరోసారి బాక్సాఫీస్ పై దండయాత్రకు సిద్ధమయ్యాడు. రవితేజ క్రాక్ చిత్రంతో మాస్ హిట్ కొట్టిన గోపిచంద్ మలినేని దర్శకత్వంలో బాలయ్య హీరోగా తెరకెక్కిన చిత్రమే వీర సింహా రెడ్డి. మైత్రీ మూవీ మేకర్స్ పతాకంపై నవీన్ ఎర్నేని, వై. రవిశంకర్ నిర్మిస్తున్న ఈ చిత్రం గురించి ఇప్పటికే అనేక ఇంట్రెస్టింగ్ వార్తలు చక్కర్లు కొడుతున్నాయి. రాయలసీమ ఫ్యాక్షన్ బ్యాక్ డ్రాప్ లో తెరకెక్కిన ఈ మూవీకి సంబంధించిన మరో ఆసక్తికరమైన సోషల్ మీడియాలో గింగిరాలు తిరుగుతోంది. అదేంటనే విషయంలోకి వెళితే.."
2,వరంగల్ ఎనుమాముల పోలీస్ స్టేషన్ ప్రారంభించిన మంత్రి ఎర్రబెల్లి.. ఆసక్తికర వ్యాఖ్యలు!!,"వరంగల్ పోలీస్ కమిషనరేట్ పరిధిలో నూతనంగా ఏర్పాటుచేసిన ఎనుమాముల పోలీస్ స్టేషన్ ను రాష్ట్ర పంచాయతీరాజ్ శాఖ మంత్రి ఎర్రబెల్లి దయాకర్ రావు నేడు ప్రారంభించారు. మంత్రి ఎర్రబెల్లి దయాకర్ రావు తోపాటు పోలీస్ స్టేషన్ ప్రారంభోత్సవ కార్యక్రమంలో ముఖ్యఅతిథిగా రాష్ట్ర స్త్రీ శిశు సంక్షేమ, గిరిజన శాఖ మంత్రి సత్యవతి రాథోడ్, వరంగల్ పోలీస్ కమిషనర్ రంగనాథ్, స్థానిక శాసనసభ్యులు ఆరూరి రమేష్, పార్లమెంటు సభ్యులు, ఇతర ప్రజా ప్రతినిధులు హాజరయ్యారు."
3,Avatar 2 Collections: అవతార్ 2 మరో సంచలనం.. 5 రోజుల్లోనే అన్ని వేల కోట్లు.. ఇక్కడ 200 కోట్లు ఔట్,"అన్ని భాషల కంటే హాలీవుడ్‌లోనే హై రేంజ్ మూవీలు వస్తుంటాయన్న విషయం తెలిసిందే. భారీ బడ్జెట్, హై టెక్నికల్ వ్యాల్యూస్, విజువల్ ఎఫెక్ట్స్‌తో అక్కడ ఎన్నో సినిమాలు వస్తుంటాయి. అలా వచ్చే చిత్రాలకు ప్రపంచ వ్యాప్తంగా భారీ స్థాయిలో స్పందన దక్కుతోంది. ఈ క్రమంలోనే ఇటీవలే వచ్చిన భారీ స్థాయి చిత్రమే 'అవతార్: ది వే ఆఫ్ వాటర్'. విడుదలకు ముందే వరల్డ్ వైడ్‌గా భారీ బజ్‌ను ఏర్పరచుకుని.. ఈ మూవీ ప్రేక్షకుల ముందుకు వచ్చింది. అనుకున్నట్లుగానే దీనికి వసూళ్ల పోటెత్తుతున్నాయి. ఈ నేపథ్యంలో 'అవతార్ 2' 5 రోజుల రిపోర్టును చూడండి!"
4,టీటీడీ ఈవో ధ‌ర్మారెడ్డి కుటుంబంలో విషాదం - మాజీ సీజేఐ ఎన్వీ రమణ ఓదార్పు..!!,తిరుమ‌ల తిరుప‌తి దేవ‌స్థానం ఈవో ధ‌ర్మారెడ్డి కుమారుడు చంద్ర‌మౌళి (28) గుండెపోటుతో కన్నుమూసారు. చెన్నైలోని కావేరి ఆస్ప‌త్రిలో వెంటిలేట‌ర్‌పై చికిత్స పొందుతూ తుది శ్వాస విడిచారు. ధర్మారెడ్డి కుమారుడు చంద్రమౌళి ముంబాయిలో ఉద్యోగం చేస్తున్నారు. సివిల్స్ కు ప్రిపేర్ అవుతున్నారు. వివాహం ఖరారు కావటంతో ఆ పనుల్లో చెన్నైకి వచ్చారు. చెన్నైకి చెందిన టీటీడీ సలహా మండలి అధ్యక్షుడు ఏజే శేఖర్ రెడ్డి కుమార్తెతో నిశ్చితార్దం జరిగింది. జనవరిలో వీరి వివాహం తిరుమలలో జరిపేందుకు నిర్ణయించారు.


In [ ]:
df.columns

Index(['Title ', 'News'], dtype='object')

In [ ]:
# combine columns , 'summary'
cols = ['Title ', 'News']
df['text'] = df[cols].apply(lambda row: ' \n '.join(row.values.astype(str)), axis=1)
df['text'].head()

0                                                                             వాల్తేరు వీరయ్యలో రవితేజ రోల్‌ లీక్: అందులో చిరంజీవినే మించేలా! \n తెలుగు సినీ ఇండస్ట్రీలో దాదాపు నలభై ఏళ్లుగా హవాను చూపిస్తూ స్టార్‌గా వెలుగొందుతోన్నారు మెగాస్టార్ చిరంజీవి. మధ్యలో కొంత కాలం గ్యాప్ ఇచ్చినా.. 'ఖైదీ నెంబర్ 150' మూవీతో సినీ రంగంలోకి రీఎంట్రీ ఇచ్చారు. అప్పటి నుంచి ఆయన వెనక్కి తిరిగి చూడకుండా వరుస సినిమాలతో దూసుకుపోతోన్నారు. ఇలా ఇప్పటికే పలు చిత్రాలతో ఈ మెగా హీరో ప్రేక్షకులను అలరించారు. ఈ క్రమంలోనే ఇప్పుడు మెగాస్టార్ చిరంజీవి 'వాల్తేరు వీరయ్య' అనే సినిమాను చేస్తున్నారు. మాస్ ఎంటర్‌టైనర్‌గా రాబోతున్న ఈ మూవీపై అంచనాలు భారీగానే ఉన్నాయి.
1    Balakrishna ఫ్యాన్స్ కు బాలయ్య మాస్ ట్రీట్.. ఒక్క టికెట్ పై ముగ్గురిని తలపించేలా డిఫరెంట్ గా! \n ఇటీవల అఖండ సినిమాతో సాలిడ్ హిట్ కొట్టిన నందమూరి నటసింహం బాలయ్య బాబు మరోసారి బాక్సాఫీస్ పై దండయాత్రకు సిద్ధమయ్యాడు. రవితేజ క్రాక్ చిత్రంతో మాస్ హిట్ కొట్టిన గోపిచంద్ మలినేని దర్శకత్వంలో బాలయ్య హీరోగా తెరకెక్కిన చిత్రమే వీర సింహా రెడ్డి. మైత్రీ మూవీ మేకర్స్ పతాకంపై నవీ

In [ ]:
# Get the embeddings
embeds = co.embed(texts=list(df['text']),model="multilingual-22-12",truncate="LEFT").embeddings

In [ ]:
# Check the dimensions of the embeddings
embeds = np.array(embeds)
print(embeds.shape)
print(embeds)
print(df['text'][0])
print(embeds[0])

In [ ]:
print(embeds.shape)

(113, 768)


In [ ]:
# Create the search index, pass the size of embedding
search_index = AnnoyIndex(embeds.shape[1], 'angular')
print(search_index)
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])
    print(search_index)


search_index.build(10) # 10 trees
search_index.save('test.ann')

In [ ]:
# Choose an example (we'll retrieve others similar to it)
example_id = 5

# Retrieve nearest neighbors
similar_item_ids = search_index.get_nns_by_item(example_id,10,
                                                include_distances=True)
# Format and print the text and distances
results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['text'], 
                             'distance': similar_item_ids[1]}).drop(example_id)

print(f"Question:'{df.iloc[example_id]['text']}'\nNearest neighbors:")
results

Question:'రచ్చకెక్కిన కోదాడ బీఆర్ఎస్ నేతల వర్గపోరు... ఎమ్మెల్యే వర్సెస్ మున్సిపల్ చైర్మన్ 
 తెలంగాణ రాష్ట్రంలో బీఆర్ఎస్ పార్టీలో వర్గ విభేదాలు చోటు చేసుకుంటున్నాయి. మొన్నటికి మొన్న మల్లారెడ్డి పై మేడ్చల్ ఎమ్మెల్యే ల తిరుగుబాటు తరువాత, జిహెచ్ఎంసి మేయర్ విజయలక్ష్మికి ఉప్పల్ ఎమ్మెల్యే బేతి సుభాష్ రెడ్డి అనుచరులు షాక్ ఇచ్చిన విషయం తెలిసిందే. ఇక తాజాగా కోదాడలోని ఎమ్మెల్యే వర్సెస్ మున్సిపల్ చైర్మన్ రచ్చ రోడ్డెక్కింది. ఇప్పటికే అనేక మార్లు ఎమ్మెల్యే బొల్లం మల్లయ్య యాదవ్, మున్సిపల్ చైర్ పర్సన్ శిరీష మధ్య అనేక సందర్భాల్లో ఘర్షణ వాతావరణం చోటు చేసుకుంది. తాజాగా మరోమారు వీరి మధ్య వివాదం చిలికి చిలికి గాలివానగా మారింది.'
Nearest neighbors:


,texts,distance
1,"Balakrishna ఫ్యాన్స్ కు బాలయ్య మాస్ ట్రీట్.. ఒక్క టికెట్ పై ముగ్గురిని తలపించేలా డిఫరెంట్ గా! \n ఇటీవల అఖండ సినిమాతో సాలిడ్ హిట్ కొట్టిన నందమూరి నటసింహం బాలయ్య బాబు మరోసారి బాక్సాఫీస్ పై దండయాత్రకు సిద్ధమయ్యాడు. రవితేజ క్రాక్ చిత్రంతో మాస్ హిట్ కొట్టిన గోపిచంద్ మలినేని దర్శకత్వంలో బాలయ్య హీరోగా తెరకెక్కిన చిత్రమే వీర సింహా రెడ్డి. మైత్రీ మూవీ మేకర్స్ పతాకంపై నవీన్ ఎర్నేని, వై. రవిశంకర్ నిర్మిస్తున్న ఈ చిత్రం గురించి ఇప్పటికే అనేక ఇంట్రెస్టింగ్ వార్తలు చక్కర్లు కొడుతున్నాయి. రాయలసీమ ఫ్యాక్షన్ బ్యాక్ డ్రాప్ లో తెరకెక్కిన ఈ మూవీకి సంబంధించిన మరో ఆసక్తికరమైన సోషల్ మీడియాలో గింగిరాలు తిరుగుతోంది. అదేంటనే విషయంలోకి వెళితే..",0.684807
2,"వరంగల్ ఎనుమాముల పోలీస్ స్టేషన్ ప్రారంభించిన మంత్రి ఎర్రబెల్లి.. ఆసక్తికర వ్యాఖ్యలు!! \n వరంగల్ పోలీస్ కమిషనరేట్ పరిధిలో నూతనంగా ఏర్పాటుచేసిన ఎనుమాముల పోలీస్ స్టేషన్ ను రాష్ట్ర పంచాయతీరాజ్ శాఖ మంత్రి ఎర్రబెల్లి దయాకర్ రావు నేడు ప్రారంభించారు. మంత్రి ఎర్రబెల్లి దయాకర్ రావు తోపాటు పోలీస్ స్టేషన్ ప్రారంభోత్సవ కార్యక్రమంలో ముఖ్యఅతిథిగా రాష్ట్ర స్త్రీ శిశు సంక్షేమ, గిరిజన శాఖ మంత్రి సత్యవతి రాథోడ్, వరంగల్ పోలీస్ కమిషనర్ రంగనాథ్, స్థానిక శాసనసభ్యులు ఆరూరి రమేష్, పార్లమెంటు సభ్యులు, ఇతర ప్రజా ప్రతినిధులు హాజరయ్యారు.",0.687418
91,"पेन को लेकर प्रेम नगर थाने में भाजपाइयों ने काटा हंगामा \n खबर उत्तर प्रदेश के बरेली (Bareilly) जिले से है। यहां प्रेम नगर थाने में उस समय हड़कंप मच गया, जब थाने के मुंशी ने बीजेपी युवा मोर्चा के पदाधिकारी को पेन देने से इनकार कर दिया। जिसके बाद युवा मोर्चा के कार्यकर्ता ने थाने में हंगामा कर दिया। इतना ही नहीं, सूचना मिलते ही कई और बीजेपी कार्यकर्ता थाने पहुंच गए और हंगामा करने लगा। यह हंगामा थाने में घंटों तक चला।",0.687796
99,"महाराष्ट्र के बामनी गांव चरवाहा बना सरपंच, जानिए कौन हैं? \n महाराष्ट्र के बामनी गांव में एक नया इतिहास लिखा गया है।यहां पर एक चरवाहा सरपंच चुना गया है वो भी भारी मतों से,इस सरपंच का नाम है प्रल्हाद बुधाजी आलाम,जिनकी उम्र 55 साल है और वो आदिवासी समाज से आते हैं,जिन्हें गांव वालों ने सरपंच चुना है। प्रल्हाद बुधाजी के लिए गांव के युवाओं ने जमकर प्रचार किया था।",0.689361
9,"Taliban: మీరు చదువుకుని ఎవరిని ఉద్దరిస్తారు ?, నోరుమూసుకుని ఇంట్లో ఉండండి, షాక్ ! \n అఫ్గానిస్తాన్ లో తాలిబన్లు వారి అరచకాలను మెల్లిమెల్లిగా కొనసాగిస్తున్నారు. ఒకేసారి ప్రతాపం చూపిస్తే ప్రజల నుంచి వ్యతిరేకత ఎదురౌతుందని భయపడిన తాలిబన్లు ఇప్పుడు వారి పైచాచికం ప్రదర్శిస్తున్నారు. తాలిబన్లు మహిళల మీద పంజా విసిరారు. మేము చెప్పిందే వేదం, మేము చెప్పినట్లు వినాలి అంటూ తాలిబన్లు ఆదేశంలోని మహిళలు కాలేజ్ లకు వెళ్లకుండా నిషేధించారు.",0.690575
6,Year Ender 2022 : ఏపీలో మారిన పొలిటికల్ సీన్-అటు మైనస్-ఇటు మాత్రం ప్లస్ కావట్లేదా ? \n ఏపీలో 2019 ఎన్నికల తర్వాత మారిన పరిస్ధితుల్లో రాష్ట్రంలో బలపడుతూ వస్తున్న వైసీపీ 2024 ఎన్నికల్లోనూ అదే ఫీట్ రిపీట్ చేసేందుకు ముమ్మరంగా ప్రయత్నిస్తోంది. అదే సమయంలో విపక్షాలు రోజుకో వ్యూహంతో జగన్ ను ఇరుకునపెట్టి రాజకీయంగా ప్రయోజనం పొందేందుకు ప్రయత్నిస్తున్నాయి. అయితే ఈ ప్రయత్నంలో ఈ ఏడాది,0.694247
23,"ఏపీ సహా 6 రాష్ట్రాల్లో బీఆర్ఎస్ కార్యకలాపాలు: ఢిల్లీ వేదికగా కేసీఆర్ దూకుడు షురూ \n ఇటీవల దేశ రాజధానిలో భారత్ రాష్ట్ర సమితి కేంద్ర కార్యాలయాన్ని ప్రారంభించిన ఆ పార్టీ అధినేత, తెలంగాణ సీఎం కేసీఆర్.. పార్టీ విస్తరణపై దృష్టి సారించినట్లు తెలుస్తోంది. డిసెంబర్ నెలాఖరు నుంచి బీఆర్ఎస్ కార్యకలాపాలు దేశ వ్యాప్తంగా పలు రాష్ట్రాల్లో ఊపందుకుంటాయని ఆ పార్టీ వర్గాలు చెబుతున్నాయి. త్వరలోనే ఢిల్లీలో పార్టీ అధినేత మకాం వేసి అక్కడే జాతీయ పార్టీల నేతలు, రైతు సంఘాల నేతలతో వరుస భేటీలు జరిపే అవకాశం ఉంది. అబ్ కీ బార్ కిసాన్ సర్కారు అన్న నినాదంతో ముందుకు పోవాలని పార్టీ ఆవిర్భావం రోజు హైదరాబాద్‌లో కేసీఆర్ ప్రకటించిన విషయం తెలిసిందే.",0.695326
67,"সৌমিত্র-সুজাতা এবার আইনি লড়াইয়ে মুখোমুখি, দলবদলে বিচ্ছেদ \n বিজেপির সাংসদ সৌমিত্র খাঁয়ের স্ত্রী সুজাতা মণ্ডল খাঁ তৃণমূলে যোগ দেওয়ার পরই বিপত্তি বাধে। সাংসদ সৌমিত্র প্রকাশ্যে গর্জে ওঠেন স্ত্রীর বিরুদ্ধে। বিষোদ্গার করে বিবাহ বিচ্ছেদের হুঁশিয়ারি দেন স্রী্যকে। তখন সুজাতা বিবাহ বিচ্ছেদে রাজি না হলও এখন নিজে থেকে মামলা করলেন বিচ্ছেন চেয়ে।",0.698185
11,"విద్యుత్ రంగంలో కేసీఆర్ పీహెచ్డీ చేస్తే.. ప్రజలపై ప్రభుత్వ బకాయిల కరెంట్ బిల్లుల భారమేల? \n విద్యుత్ రంగంపై సీఎం కేసీఆర్ కు ఉన్నంత పట్టు దేశంలో మరే ఇతర నాయకులకు లేదని, విద్యుత్తు రంగంలో ఆయన పీహెచ్డీ చే

In [ ]:
# query = "skin care ayurveda"
# query = "how much money did skin care ayurveda raise"
# query = "semelso wife arrest"
# query = "avatar 2 movie collection"
# query = "బాలయ్య మాస్ ట్రీట్"

def multilingual_semantic_search(query):
  # query = "is messi the best footballer of all time?"

  # Get the query's embedding
  query_embed = co.embed(texts=[query],
                    model="multilingual-22-12",
                    truncate="LEFT").embeddings

  # Retrieve the nearest neighbors
  similar_item_ids = search_index.get_nns_by_vector(query_embed[0],10,
                                                  include_distances=True)
  # Format the results
  # results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['text'], 
  #                              'distance': similar_item_ids[1]})

  results = pd.DataFrame(data={'title': df.iloc[similar_item_ids[0]]['Title '], 
                              'news': df.iloc[similar_item_ids[0]]['News'],
                              'distance': similar_item_ids[1]})

  response = {}

  # JSON response
  # for i in similar_item_ids[0]:
  #   # print(i)
  #   response[i] = \
  #             {     \
  #                 "title": df.iloc[i]['Title '], \
  #                 "news": df.iloc[i]['News'] 
  #             }

  response = """ """
  for i in similar_item_ids[0]:
  # print(i)
    response += "Title: " + df.iloc[i]['Title '] + " \n " +"Short News: "+ df.iloc[i]['News'] + "\n\n"

  # print(similar_item_ids)
  # print(similar_item_ids[0])
  # print(similar_item_ids[1])

  # print(f"Query:'{query}'\nNearest neighbors:")
  # print(results)
  # print("----------------------")
  # print(type(response))

  print(response)
  return response

In [ ]:
multilingual_semantic_search("is messi the best footballer of all time?")

 Title: People who don't consider Messi the best will keep believing that: Iniesta 
 Short News: Lionel Messi's ex-Barcelona teammate Andres Iniesta said people who don't consider Messi as the best footballer will continue to believe so despite Argentina's triumph in 2022 World Cup. "[They'll] find an excuse to...[see] it that way," added Iniesta. "If there's a single idiot that still doubts it, Messi's the best of all time," Argentina midfielder Rodrigo De Paul recently stated.

Title: Messi said 'If we win 2022 WC, I'll keep playing till next WC': Valdano 
 Short News: Following Argentina's triumph in 2022 FIFA World Cup, ex-Argentina footballer Jorge Valdano said Lionel Messi had told him that he'll keep playing for Argentina till the 2026 World Cup if they win the 2022 World Cup. "Football has shown that it's practically impossible to play...six World Cups," added Valdano. Messi has represented Argentina in five World Cups so far.

Title: ये स्मार्टफोन बहुत जल्द होने वाले हैं लांच 

' Title: People who don\'t consider Messi the best will keep believing that: Iniesta \n Short News: Lionel Messi\'s ex-Barcelona teammate Andres Iniesta said people who don\'t consider Messi as the best footballer will continue to believe so despite Argentina\'s triumph in 2022 World Cup. "[They\'ll] find an excuse to...[see] it that way," added Iniesta. "If there\'s a single idiot that still doubts it, Messi\'s the best of all time," Argentina midfielder Rodrigo De Paul recently stated.\n\nTitle: Messi said \'If we win 2022 WC, I\'ll keep playing till next WC\': Valdano \n Short News: Following Argentina\'s triumph in 2022 FIFA World Cup, ex-Argentina footballer Jorge Valdano said Lionel Messi had told him that he\'ll keep playing for Argentina till the 2026 World Cup if they win the 2022 World Cup. "Football has shown that it\'s practically impossible to play...six World Cups," added Valdano. Messi has represented Argentina in five World Cups so far.\n\nTitle: ये स्मार्टफोन बहुत जल्द

In [ ]:
!pip install gradio
import gradio as gr
# demo = gr.Interface(fn=multilingual_semantic_search, inputs="text", outputs="text")
with gr.Blocks() as demo:
    gr.Markdown("🌍 This app uses a multilingual semantic model from COhere to 🚀 revolutionize the media and news industry in multilingual markets like India, allowing anyone to track 📰 regional news in real-time without the need for translation or understanding of other regional languages. 🙌")
    name = gr.Textbox(label="*Semantic search enable! Search for a news...")
    output = gr.Textbox(label="Semantic search results")
    greet_btn = gr.Button("Search")
    theme="darkpeach"
    greet_btn.click(fn=multilingual_semantic_search, inputs=name, outputs=output)
demo.launch()  

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

In [ ]:
#@title Plot the archive {display-mode: "form"}

# UMAP reduces the dimensions from 1024 to 2 dimensions that we can plot
reducer = umap.UMAP(n_neighbors=20) 
umap_embeds = reducer.fit_transform(embeds)
# Prepare the data to plot and interactive visualization
# using Altair
df_explore = pd.DataFrame(data={'text': df['text']})
df_explore['x'] = umap_embeds[:,0]
df_explore['y'] = umap_embeds[:,1]

# Plot
chart = alt.Chart(df_explore).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    tooltip=['text']
).properties(
    width=700,
    height=400
)
chart.interactive()

alt.Chart(...)